# Vgg16 finetune

### Base imports

In [1]:
from __future__ import print_function, division

import os
from os.path import join as pj
import shutil
from glob import glob

import numpy as np
np.random.seed = 0  # for reproducibility

import pandas as pd

import matplotlib
%matplotlib inline
from matplotlib import pylab as plt
# %config InlineBackend.figure_format = 'retina'

from matplotlib.patches import Circle
import matplotlib.patheffects as PathEffects

import seaborn as sns

from PIL import Image

import json

from tqdm import tqdm_notebook as tqdm

### DL imports

In [2]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from keras.models import Model
from keras import metrics
from keras.callbacks import ModelCheckpoint, TensorBoard

Using TensorFlow backend.


### Constants

In [3]:
IMAGE_HEIGHT, IMAGE_WIDTH = 224, 224

In [4]:
TRAIN_DATA_DIR = './data/images_by_class/'
N_CHANNELS = 3
N_CLASSES = 2

In [5]:
MODEL_NAME = 'vgg16'

CHECKPOINTS_DIR = './checkpoints_{}'.format(MODEL_NAME)
if not os.path.exists(CHECKPOINTS_DIR):
    os.mkdir(CHECKPOINTS_DIR)

In [6]:
TESORBOARD_DIR = './tensorboard'
if not os.path.exists(TESORBOARD_DIR):
    os.mkdir(TESORBOARD_DIR)

In [7]:
BATCH_SIZE = 16

### Handy functions

In [8]:
def plot_image(image):
    fig, ax = plt.subplots()
    
    ax.imshow(image)
    ax.axis('off')

### Building model

In [9]:
def build_vgg16_model(image_height, image_width, n_channels, n_classes, lr=0.001):
    base_model = applications.VGG16(weights='imagenet', include_top=False,
                                    input_shape=(image_height, image_width, n_channels))

    top_model = Sequential()
    top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
    top_model.add(Dense(256, activation='relu'))
    top_model.add(Dropout(0.5))
    top_model.add(Dense(n_classes, activation='softmax'))

    model = Model(inputs=base_model.input, outputs=top_model(base_model.output))
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(lr=lr),
                  metrics=['categorical_crossentropy'])
    
    return model

In [10]:
model = build_vgg16_model(IMAGE_HEIGHT, IMAGE_WIDTH, N_CHANNELS, N_CLASSES)

### Data generators

In [11]:
train_idg = ImageDataGenerator(
    rescale=1 / 255,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True
)

In [12]:
train_generator = train_idg.flow_from_directory(
    TRAIN_DATA_DIR,
    target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

Found 29733 images belonging to 2 classes.


### Callbacks

In [13]:
callbacks = []

In [14]:
model_checkpoint = ModelCheckpoint(pj(CHECKPOINTS_DIR, 'checkpoint-{epoch:02d}.hdf5'), verbose=1)
callbacks.append(model_checkpoint)

In [15]:
tensorboard = TensorBoard(log_dir=TESORBOARD_DIR,
                           histogram_freq=0,
                           write_graph=True,
                           write_images=True,
                           batch_size=1)
callbacks.append(tensorboard)

### Learn model

In [16]:
model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=100,
    callbacks=callbacks)

Epoch 1/100
  7/100 [=>............................] - ETA: 6476s - loss: 3.7800 - categorical_crossentropy: 3.7800

KeyboardInterrupt: 